# Using the Frenet coordinate system to associate the trajectories

The input file for this comes from [../radar/lane_classification.ipynb](../radar/lane_classification.ipynb)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autoreload
%autoreload 2

# find the root of the project
import os
from pathlib import Path

ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
import sys

sys.path.append(str(ROOT))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import dotenv

# load the environment variables
dotenv.load_dotenv(ROOT.joinpath(".env"))

True

## Load the _Already Processed_ Radar Data


In [4]:
from src.radar import Filtering


f = Filtering(
    network_boundary_path=ROOT / "geo_data" / "network_outline.geojson",
    radar_location_path=ROOT / "geo_data" / "calibrated_origins.json",
)

In [5]:
import polars as pl

radar_df = pl.read_parquet(
    ROOT / "tmp" / "all_working_processed_1Lane.parquet",
)

radar_df.shape

(3978783, 39)

In [6]:
from datetime import timedelta
import numpy as np

radar_df = (
    radar_df.lazy()
    .filter(
        (pl.col("s").is_not_null())
        & (pl.col("lane").is_not_null())
        & (pl.col("epoch_time") < (pl.col("epoch_time").min() + timedelta(minutes=120)))
    )
    .sort("epoch_time", descending=True)
    .set_sorted(["epoch_time"])
    ## ---------------------------------------------------
    ## Add Lane Group
    ## ---------------------------------------------------
    # .pipe(f.add_lane_group)
    # ---------------------------------------------------
    ## Remove all of the Radar's Internal Predictions
    ## ---------------------------------------------------
    .filter(pl.col("ui16_predictionCount") < 1)
    # .with_columns(
    #     [
    #         (pl.col("epoch_time").diff() / 1000)
    #         .backward_fill()
    #         .over(["object_id", "lane"])
    #         .alias("time_diff"),
    #     ]
    # )
    ## ---------------------------------------------------
    ## Calculating the S and D components of the velocity
    ## ---------------------------------------------------
    .with_columns(
        [
            (pl.col("direction") - pl.col("s_angle")).sin().alias("sin"),
            (pl.col("direction") - pl.col("s_angle")).cos().alias("cos"),
        ]
    )
    .pipe(f.atan2, x_col="cos", y_col="sin", out_col="s_angle_diff", normalize=False)
    .with_columns(
        pl.when(pl.col("s_angle_diff").abs() > np.deg2rad(30))
        .then(None)
        .otherwise(pl.col("s_angle_diff"))
        .alias("s_angle_diff")
    )
    .with_columns(
        [
            pl.col("s_angle_diff")
            .interpolate()
            .over(["object_id", "lane"])
            .alias("s_angle_diff")
        ]
    )
    .filter(pl.col("s_angle_diff").is_not_null())
    .with_columns(
        [
            (pl.col("f32_velocityInDir_mps") * pl.col("s_angle_diff").cos()).alias(
                "s_velocity"
            )
            * -1,
            (pl.col("f32_velocityInDir_mps") * pl.col("s_angle_diff").sin()).alias(
                "d_velocity"
            )
            # * -1,
        ]
    )
    ## ---------------------------------------------------
    ## Map the distance to the front and back of the
    ##          vehicle to the S dimension
    ## ---------------------------------------------------
    .with_columns(
        [
            (pl.col("f32_distanceToFront_m") * pl.col("s_angle_diff").cos()).alias(
                "distanceToFront_s"
            ),
            (pl.col("f32_distanceToBack_m") * pl.col("s_angle_diff").cos()).alias(
                "distanceToBack_s"
            ),
            # do the vehicle length
            (pl.col("f32_length_m") * pl.col("s_angle_diff").cos()).alias("length_s"),
        ]
    )
    ## ---------------------------------------------------
    ## Making a Z (measurement) vector for the Kalman Filter (+ time)
    ## ---------------------------------------------------
    .with_columns(
        [
            pl.concat_list(
                [
                    pl.col("s"),
                    pl.col("s_velocity"),
                    pl.col("min_d"),
                    pl.col("d_velocity"),
                ]
            ).alias("z")
        ]
    )
    .collect()
)

function: atan2 took: 0.00043511390686035156 seconds


### Mark Whether A Trajectory Is Approaching Radar or Not


In [7]:
from src.geometry import get_radar_origins

locations = get_radar_origins(ROOT / "geo_data" / "calibrated_origins.json")
locations["utm_x"] = locations["geometry"].x
locations["utm_y"] = locations["geometry"].y

locations = locations.reset_index(names="ip")[["ip", "utm_x", "utm_y"]]

locations_df = pl.DataFrame(locations)

radar_df = (
    radar_df.lazy()
    .join(locations_df.lazy(), on="ip", suffix="_radar")
    .collect(streaming=True)
)

radar_df = (
    radar_df.lazy()
    .sort("epoch_time")
    .with_columns(
        [
            (
                (pl.col("x") - pl.col("utm_x_radar")) ** 2
                + (pl.col("y") - pl.col("utm_y_radar")) ** 2
            )
            .sqrt()
            .alias("distance_to_radar")
        ]
    )
    .with_columns(
        [
            # get the direction to the radar by diffing the distance on object id level
            pl.col("distance_to_radar")
            .diff()
            .backward_fill()
            .over("object_id")
            .alias("distance_to_radar_diff"),
        ]
    )
    .with_columns(
        [
            (pl.col("distance_to_radar_diff") < 0).alias("towards_radar"),
        ]
    )
    .collect(streaming=True)
)

### Filter Short Trajectories


In [8]:
radar_df = (
    radar_df.with_columns(
        (pl.col("epoch_time").diff() / 1000)
        .sum()
        .over(
            [
                "object_id",
                "lane",
            ]
        )
        .alias("lane_time_diff"),
        (pl.col("s").diff() * -1)
        .sum()
        .over(
            [
                "object_id",
                "lane",
            ]
        )
        .alias("lane_distance"),
    )
    # gotta be on the lane for 5 seconds & travel for 10 meters
    .filter((pl.col("lane_time_diff") > 5) & (pl.col("lane_distance") > 10)).drop(
        ["lane_time_diff"]
    )
)

### Label Lane Trajectories As Needing Extension or Not

Need an extension if the vehicle ends near a lane center. Otherwise we assume that it has left the netwerk


In [9]:
# aka do the ends need a match or not
radar_df = (
    radar_df.with_columns(
        [
            pl.col("epoch_time")
            .max()
            .over(
                [
                    "object_id",
                ]
            )
            .alias("max_time_vehicle"),
            pl.col("epoch_time")
            .min()
            .over(
                [
                    "object_id",
                ]
            )
            .alias("min_time_vehicle"),
        ]
    )
    .with_columns(
        [
            (pl.col("epoch_time") == pl.col("max_time_vehicle"))
            .over(["object_id"])
            .alias("is_end"),
            (pl.col("epoch_time") == pl.col("min_time_vehicle"))
            .over(["object_id"])
            .alias("is_start"),
            pl.col("min_d").last().over(["object_id"]).alias("last_d"),
        ]
    )
    .drop(["max_time_vehicle", "min_time_vehicle"])
)

In [10]:
radar_df = radar_df.sort(
    by=["epoch_time"],
).with_columns(
    (pl.col("is_end").any() & pl.col("last_d").is_between(-2, 5))
    .over(["object_id", "lane"])
    .alias("extend")
)

### Build Prediction Extension onto the DataFrame


In [11]:
extend_df = (
    radar_df.groupby(
        [
            "object_id",
            "lane",
        ]
    )
    .agg([pl.col("extend").any().alias("extend_me"), pl.all().last()])
    .filter(pl.col("extend_me"))
)

extend_df = extend_df.extend(
    extend_df.with_columns([(pl.col("epoch_time") + timedelta(seconds=4))])
)

extend_df = (
    extend_df.sort(by="epoch_time")
    .set_sorted("epoch_time")
    .drop(["extend_me"])
    .upsample(
        time_column="epoch_time",
        every="100ms",
        by=[
            "object_id",
            "lane",
        ],
    )
    .with_columns(pl.all().forward_fill())
    .with_columns([pl.lit(None, dtype=pl.List(pl.Float64())).alias("z")])
)

radar_df = (
    radar_df.with_columns(
        [
            pl.lit(False).alias("prediction"),
        ]
    )
    .vstack(
        extend_df.select(radar_df.columns).with_columns(
            [
                pl.lit(True).alias("prediction"),
            ]
        )
    )
    .unique(["object_id", "epoch_time"])
    .sort("epoch_time")
    .set_sorted("epoch_time")
)

### Upsample the DataFrame to every 100ms

edit, this doesn't work well, because the data is not evenly spaced in time. I need to do something else.


In [12]:
radar_df = radar_df.sort(
    by=["epoch_time"],
).with_columns(
    [
        (pl.col("epoch_time").diff() / 1000)
        .backward_fill(1)
        .over(
            "object_id",
        )
        .fill_null(0)
        .alias("time_diff")
    ]
)

In [13]:
# TODO:

# 1. split trajectories that have dt > 4? seconds into different trajectories (done by changing the object id)
radar_df = (
    radar_df.with_columns(
        (pl.col("time_diff") > 4).alias("split"),
        pl.col("object_id").cast(str).alias("object_id"),
    )
    .with_columns(
        pl.col("split")
        .cumsum()
        .over(["object_id", "lane"])
        .cast(str)
        .alias("trajectory_id"),
    )
    .with_columns(
        pl.concat_str(
            [pl.col("object_id"), pl.col("trajectory_id")], separator="-"
        ).alias("object_id"),
    )
    .with_columns(pl.col("object_id").cast(pl.Categorical).cast(int))
)

In [14]:
test_df = (
    radar_df.lazy()
    .select(["object_id", "epoch_time", "z", "time_diff", "prediction"])
    .with_columns(
        # calculate the time "index" for each vehicle, aka just a counter
        [
            pl.lit(1).cast(pl.Int32).alias("time_ind"),
        ]
    )
    .with_columns(
        [
            (pl.col("time_ind").cumsum() - 1)
            .over(
                [
                    "object_id",
                ]
            )
            .alias("time_ind"),
        ]
    )
)


test_df = (
    test_df.join(
        test_df.groupby("object_id")
        .agg(pl.col("time_ind").max())
        .sort("time_ind")
        .with_row_count("vehicle_ind"),
        on="object_id",
        how="inner",
    )
    .sort(["vehicle_ind", "time_ind"])
    .with_columns(pl.col("vehicle_ind").cast(pl.Int32).alias("vehicle_ind"))
    .collect()
)

In [15]:
from src.filters.kalman import kf_filter_lanechange

import torch

# list gpu devices
torch.cuda.device_count()

1

In [16]:
print("Dimensions", test_df["vehicle_ind"].max(), "x", test_df["time_ind"].max())

Dimensions 16921 x 737


In [17]:
# %%timeit -n 2 -r 1

# x_filt, p_filt = apply_filter(
#     test_df,
# )

# t_index = np.repeat(np.arange(x_filt.shape[0]), x_filt.shape[1])
# v_index = np.tile(np.arange(x_filt.shape[1]), x_filt.shape[0])
# x_filt = x_filt.reshape(-1, 6)
# p_filt = p_filt.reshape(-1, 6, 6)

In [18]:
# # %%timeit -n 2 -r 1

# x_filt, p_filt = apply_filter(
#     test_df,
#     gpu=True
# )

# # flush the gpu memory
# torch.cuda.empty_cache()

# t_index = np.repeat(np.arange(x_filt.shape[0]), x_filt.shape[1])
# v_index = np.tile(np.arange(x_filt.shape[1]), x_filt.shape[0])
# x_filt = x_filt.reshape(-1, 6)
# p_filt = p_filt.reshape(-1, 6, 6)

In [19]:
torch.cuda.empty_cache()

In [20]:
# from src.filters.vectorized_kalman import CALKFilter, CALCFilter

# torch.cuda.empty_cache()

# kf = CALCFilter(
#     test_df,
# )
# x_filt, p_filt = kf.apply_filter()
# t_index = np.repeat(np.arange(kf.t_dim), kf.v_dim)
# v_index = np.tile(np.arange(kf.v_dim), kf.t_dim)
# x_filt = x_filt.reshape(-1, kf.x_dim)
# p_filt = p_filt.reshape(-1, kf.x_dim, kf.x_dim)

In [21]:
from src.filters.vectorized_kalman import IMMFilter
import gc


if 'kf' in globals():
    # ignore the warning
    del kf
    gc.collect()
    torch.cuda.empty_cache()


kf = IMMFilter(
    test_df,
    filters=("CALC", "CALK", "CVLK"),
    M=np.array([[0.95, 0.05, 0.05], [0.05, 0.95, 0.05], [0.05, 0.05, 0.95]]),
    mu=np.array([0.02, 0.5, 0.48]),
)

x_filt, p_filt = kf.apply_filter()
mu = kf._mu.detach().cpu().numpy()

100%|██████████| 738/738 [00:09<00:00, 76.89it/s] 


In [22]:
# x_filt, p_filt = kf._filters[0].x.detach().numpy(), kf._filters[0].P.detach().numpy()
t_index = np.repeat(np.arange(kf.t_dim), kf.v_dim)
v_index = np.tile(np.arange(kf.v_dim), kf.t_dim)
x_filt = x_filt.reshape(-1, kf.x_dim)
p_filt = p_filt.reshape(-1, kf.x_dim, kf.x_dim)
mu = mu.reshape(-1, kf.f_dim)

In [23]:
# %%timeit -n 2 -r 1
# # from src.filters.kalman2 import kf_filter_lanechange

# # trying the apply approach. Way uglier but 2x faster on my machine?
# # Faster than all this is a vectorized approach, but not as clean
# radar_df = (
#     test_df
#     .lazy()
#     .with_columns([pl.struct(["time_diff", "z", "prediction"]).alias("filter_input")])
#     .sort(["epoch_time"])
#     .groupby(
#         [
#             "object_id",
#         ]
#     )
#     .agg(
#         [
#             pl.col("filter_input")
#             .apply(
#                 kf_filter_lanechange,
#             )
#             .alias("filter_output"),
#             # have to keep the time for the resulting join
#             pl.col("epoch_time"),
#         ]
#     )
#     .explode(["filter_output", "epoch_time"])
#     .unnest("filter_output")
#     .collect(streaming=True)
# )

In [23]:
# create a dataframe of the filtered states
filt_df = pl.DataFrame(
    {
        "time_ind": t_index,
        "vehicle_ind": v_index,
        "s": x_filt[:, 0],
        "s_velocity": x_filt[:, 1],
        "s_accel": x_filt[:, 2],
        "d": x_filt[:, 3],
        "d_velocity": x_filt[:, 4],
        "d_accel": x_filt[:, 5],
        "P": p_filt,
        "calc_p": mu[:, 0],
        "calk_p": mu[:, 1],
        "cvlk_p": mu[:, 2],
    }
).with_columns(
    [
        pl.col("time_ind").cast(pl.Int32),
        pl.col("vehicle_ind").cast(pl.Int32),
    ]
)


# join the filtered states to the original data
joined_df = (
    test_df.lazy()
    .with_columns(
        [
            # # unpack the list of z values
            pl.col("z").list.get(0).alias("s"),
            pl.col("z").list.get(1).alias("s_velocity"),
            pl.col("z").list.get(2).alias("d"),
            pl.col("z").list.get(3).alias("d_velocity"),
        ]
    )
    .join(filt_df.lazy(), on=["time_ind", "vehicle_ind"], how="inner", suffix="_filt")
    .collect()
)

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# from src.filters.kalman2 import kf_filter_lanechange

# veh = joined_df['object_id'].sample(1).to_numpy()[0]
veh = 2998

veh_df = joined_df.filter(pl.col("vehicle_ind") == veh).sort("epoch_time")

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=(
        f"Vehicle {veh} S",
        f"Vehicle {veh} D",
    ),
    # add a secondary y axis to the velocity plots
    specs=[
        [{"secondary_y": True}],
        [{"secondary_y": True}],
        [{"secondary_y": False}],
    ],
)


colors = {
    "": "blue",
    "_filt": "red",
}


for df, ext in [(veh_df, ""), (veh_df, "_filt")]:
    fig.add_trace(
        go.Scatter(
            x=veh_df["epoch_time"],
            y=df[f"s{ext}"],
            mode="markers+lines",
            name=f"S{ext}",
            marker_color=colors[ext],
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=veh_df["epoch_time"],
            y=df[f"s_velocity{ext}"] * -1,
            mode="markers+lines",
            name=f"S Velocity{ext}",
            marker_color=colors[ext],
        ),
        row=1,
        col=1,
        secondary_y=True,
    )

    # add the D dimension
    fig.add_trace(
        go.Scatter(
            x=veh_df["epoch_time"],
            y=df[f"d{ext}"],
            mode="markers+lines",
            name=f"D{ext}",
            marker_color=colors[ext],
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=veh_df["epoch_time"],
            y=df[f"d_velocity{ext}"],
            mode="markers+lines",
            name=f"D Velocity{ext}",
            marker_color=colors[ext],
        ),
        row=2,
        col=1,
        secondary_y=True,
    )


for p in ["calc_p", "calk_p", "cvlk_p"]:
    # plot the probabilities
    fig.add_trace(
        go.Scatter(
            x=veh_df["epoch_time"],
            y=veh_df[p],
            mode="markers+lines",
            name=p,
            # marker_color="green",
        ),
        row=3,
        col=1,
    )


# bound the y axis
# fig.update_yaxes(range=[-10, 100], row=1, col=1)
fig.update_yaxes(range=[-10, 10], row=2, col=1)

fig.update_layout(
    height=800,
    width=1200,
)

### Debugging by Implementing the Same Filter with FilterPy

In [37]:
from filterpy.kalman import KalmanFilter
from filterpy.kalman import IMMEstimator

In [ ]:
def F(mode='cvlk'):
    
    